In [1]:
import numpy as np
import pandas as pd
import datetime
import pyupbit
import importlib

In [2]:
# trading_helper 설치 및 버전 확인

!wget https://esohn.be/python/trading_helper.py -O trading_helper.py

import trading_helper
import importlib
importlib.reload(trading_helper)

print( f'Trading Helper Version: {trading_helper.TRADING_HELPER_VERSION}'  )

--2023-06-08 20:23:42--  https://esohn.be/python/trading_helper.py
Resolving esohn.be (esohn.be)... 18.64.8.126, 18.64.8.120, 18.64.8.8, ...
Connecting to esohn.be (esohn.be)|18.64.8.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6252 (6.1K) [text/x-python]
Saving to: ‘trading_helper.py’

trading_helper.py   100%[===================>]   6.11K  --.-KB/s    in 0s      

2023-06-08 20:23:42 (139 MB/s) - ‘trading_helper.py’ saved [6252/6252]

Trading Helper Version: 2023.05.15-v1


In [3]:
# ------------------------------------------------------------------------------------------------------
# https://esohn.be/account 의 AUTH CODE 로 실행 로그 기록
# ------------------------------------------------------------------------------------------------------

'''
AUTH_CODE = 'f587ea651febd5446efc61d5e3fef123'  -> 아래에 이렇게 auth code를 붙여넣으세요.
'''
AUTH_CODE = '2b435454dd9860ceba97f35f7aa030f0'

# Do not edit
trading_helper.set_auth_code( AUTH_CODE )


Log updated! (2023-1 YCS1009-01 박희영)


In [8]:
from_date = '20220501'
to_date = '20230602'

df = trading_helper.get_multiple(from_date, to_date)


df

,Unnamed: 0,index,price_btc,price_eth,price_doge,price_ada,bal_krw,bal_btc,bal_eth,bal_doge,...,bal2_ada,target_btc,target_eth,target_doge,target_ada,trade_btc,trade_eth,trade_doge,trade_ada,value
0,0,2022-05-01 09:00:00,49283000.0,3570000.0,166.0,996.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2022-05-02 09:00:00,49383000.0,3625000.0,170.0,1010.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,2022-05-03 09:00:00,49501000.0,3667000.0,168.0,1000.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,2022-05-04 09:00:00,48772000.0,3595000.0,167.0,997.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,2022-05-05 09:00:00,50631000.0,3752000.0,173.0,1140.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,392,2023-05-28 09:00:00,35824000.0,2445000.0,96.7,489.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
393,393,2023-05-29 09:00:00,37360000.0,2540000.0,97.8,509.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
394,394,2023-05-30 09:00:00,36992000.0,2522000.0,97.5,504.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
395,395,2023-05-31 09:00:00,36923000.0,2529000.0,96.5,502.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# ------------------------------------------------------------------------------------------------------
# - 코드를 완성했다면 이 코드셀만 복사해서 competition 페이지 Rebalancing 탭에 라운드 별 코드 항목으로 붙여넣습니다.
# - 연습할 때는 다음과 같은 이유로 프로젝트 페이지의 에디터에서 직접 연습하지 말고 Colab에서 연습하세요. 
#   + Colab의 연습은 서버에 횟수가 기록되어 평가에 반영됩니다.
#   + Colab 노트북 자체에도 변경 history가 누적 됩니다. (부정행위 방지)
#   + 프로젝트 페이지에서 submit 횟수가 지나치게 빈번한 경우 submit이 일정시간 제한될 수 있습니다.
# ------------------------------------------------------------------------------------------------------

# Given Code Version 2023.05.16-v2

import numpy as np
import pandas as pd
import datetime

# rebalancing() 암호화폐 포트폴리오 리밸런싱 함수
'''
* ratio_  자산배분 비율 (총 합은 반드시 1.0이어야 함)
  투자하지 않는 암호화폐는 0으로 남겨둠
  
  예) 원화 10%, 비트코인(BTC) 50%, 이더리움(ETH) 40%
      ratio_krw = 0.1
      ratio_btc = 0.5
      ratio_eth = 0.4
      ratio_doge = 0.0
      ratio_ada = 0.0

  예) 원화 70%, 비트코인(BTC) 10%, 이더리움(ETH) 10%, 도지코인(DOGE) 5%, 에이다(ADA) 5%
      ratio_krw = 0.7
      ratio_btc = 0.1
      ratio_eth = 0.1
      ratio_doge = 0.05
      ratio_ada = 0.05

* threshold - 최소한으로 요구되는 총 자산 대비 변동 비율 (%)

  기본값 0: 조금이라도 변동될 때 매매
  예) 0.001: 전체 자산의 0.1% 변동시 매매, 0.005: 전체 자산의 0.5% 변동시 매매


* fee - 수수료, 기본값 0.001 (변경 없음)

* init_krw - 초기 투자금액, 기본값 100000000 (변경 없음)
'''

# Qualification 라운드 에서는 지정된 암호화폐 비율과 threshold 0.0을 사용해야 하며
# 그 외의 라운드에서는 암포화폐 비율과 threshold를 자유롭게 사용할 수 있음

# ------------------------------------------------
# EDITABLE (1)
#
DEFAULT_RATIO_KRW = 1.0
DEFAULT_RATIO_BTC = 0.0
DEFAULT_RATIO_ETH = 0.0
DEFAULT_RATIO_DOGE = 0.0
DEFAULT_RATIO_ADA = 0.0

DEFAULT_THRESHOLD = 0.0 # Qualification 라운드 에서는 이 값을 0.0으로
# ------------------------------------------------

def rebalancing( df, ratio_krw=DEFAULT_RATIO_KRW, ratio_btc=DEFAULT_RATIO_BTC, ratio_eth=DEFAULT_RATIO_ETH, ratio_doge=DEFAULT_RATIO_DOGE, ratio_ada=DEFAULT_RATIO_ADA, threshold=DEFAULT_THRESHOLD, fee=0.001, init_krw=100000000 ):

    # -------------------------------------------------------------------------------
    # DO NOT EDIT
    #
    ratio_sum = ratio_krw + ratio_btc + ratio_eth + ratio_doge + ratio_ada
    if abs(ratio_sum - 1.0) > 0.0001:
        print(f'The sum of the ratios must be 1. ({ratio_sum})')
        return
    # -------------------------------------------------------------------------------

    row_index = 0

    # 데이터프레임의 모든 행에 대해서 반복
    for row_index in range( len( df ) ):

        # 현재 데이터: cur_data
        cur_data = df.loc[ row_index, : ].copy()
        # 이전 데이터: prev_data
        if row_index > 0:
            prev_data = df.loc[ row_index-1, : ].copy()



        #------------------------
        # Balance 계산 (bal_)
        # 하루 전의 계산 후 잔고(bal2_)를 현재의 계산 전 잔고로 가져옴(bal_)
        if row_index == 0:
            # 첫째 날이라면 잔고는 krw만 존재
            cur_data[ 'bal_krw' ] = init_krw
        else:
            # 둘째 날 부터는 이전 날 계산 후 잔고(bal2_)를 현재 잔고(bal_)로 대입
            cur_data[ 'bal_krw' ] = prev_data[ 'bal2_krw' ]
            cur_data[ 'bal_btc' ] = prev_data[ 'bal2_btc' ]
            cur_data[ 'bal_eth' ] = prev_data[ 'bal2_eth' ]
            cur_data[ 'bal_doge' ] = prev_data[ 'bal2_doge' ]
            cur_data[ 'bal_ada' ] = prev_data[ 'bal2_ada' ]
            

        #------------------------
        # Total Value 계산 (value)
        # 전체 자산의 가치를 모두 합산
        cur_data[ 'value' ] = cur_data[ 'bal_krw' ]
        cur_data[ 'value' ] += cur_data[ 'price_btc' ] * cur_data[ 'bal_btc' ]
        cur_data[ 'value' ] += cur_data[ 'price_eth' ] * cur_data[ 'bal_eth' ]
        cur_data[ 'value' ] += cur_data[ 'price_doge' ] * cur_data[ 'bal_doge' ] # EDITABLE (2)
        cur_data[ 'value' ] += cur_data[ 'price_ada' ] * cur_data[ 'bal_ada' ] # EDITABLE (3)                                
                                

        #------------------------
        # Target 계산 (target_)
        # 비율에 따라 다시 조정되어야 할 목표 잔고 계산
        cur_data[ 'target_btc' ] = ratio_btc * cur_data[ 'value' ] / cur_data[ 'price_btc' ]
        cur_data[ 'target_eth' ] = ratio_eth * cur_data[ 'value' ] / cur_data[ 'price_eth' ]
        cur_data[ 'target_doge' ] = ratio_doge * cur_data[ 'value' ] / cur_data[ 'price_doge' ]    # EDITABLE (4)
        cur_data[ 'target_ada' ] = ratio_ada * cur_data[ 'value' ] / cur_data[ 'price_ada' ]      # EDITABLE (5)


        #------------------------
        # Trading Amount 계산 (trade_)
        # Target 잔고를 위해 매매해야 할 양을 계산
        cur_data[ 'trade_btc' ] = cur_data[ 'target_btc' ] - cur_data[ 'bal_btc' ]
        cur_data[ 'trade_eth' ] = cur_data[ 'target_eth' ] - cur_data[ 'bal_eth' ]
        cur_data[ 'trade_doge' ] = cur_data[ 'target_doge' ] - cur_data[ 'bal_doge' ]  # EDITABLE (6)
        cur_data[ 'trade_ada' ] = cur_data[ 'target_ada' ] - cur_data[ 'bal_ada' ]  # EDITABLE (7)



        #------------------------
        # 매매해야 할 양의 절대값이 총 자산의 일정 %를 넘지 않으면 매매하지 않음
        th_value = threshold * cur_data[ 'value' ]
        amount_btc = abs( cur_data[ 'trade_btc' ] * cur_data[ 'price_btc' ] )
        amount_eth = abs( cur_data[ 'trade_eth' ] * cur_data[ 'price_eth' ] )
        amount_doge = abs( cur_data[ 'trade_doge' ] * cur_data[ 'price_doge' ] )  # EDITABLE (8)
        amount_ada = abs( cur_data[ 'trade_ada' ] * cur_data[ 'price_ada' ] )  # EDITABLE (9)


        #--------------------------------------------------------------------------------------------------------------------
        # DO NOT EDIT
        # 계산 후 잔고 준비
        cur_data[ 'bal2_krw' ] = cur_data[ 'bal_krw' ]
        cur_data[ 'bal2_btc' ] = cur_data[ 'bal_btc' ]
        cur_data[ 'bal2_eth' ] = cur_data[ 'bal_eth' ]
        cur_data[ 'bal2_doge' ] = cur_data[ 'bal_doge' ]
        cur_data[ 'bal2_ada' ] = cur_data[ 'bal_ada' ]

        if amount_btc >= th_value:
            # BTC 잔고 변경 
            cur_data[ 'bal2_btc' ] += cur_data[ 'trade_btc' ]
            # krw 잔고 변경 (BTC 매매 반영 + 수수료 차감)
            cur_data[ 'bal2_krw' ] -= cur_data[ 'trade_btc' ] * cur_data[ 'price_btc' ] + amount_btc * fee

        if amount_eth >= th_value:
            # ETH 잔고 변경 
            cur_data[ 'bal2_eth' ] += cur_data[ 'trade_eth' ]
            # krw 잔고 변경 (ETH 매매 반영 + 수수료 차감)
            cur_data[ 'bal2_krw' ] -= cur_data[ 'trade_eth' ] * cur_data[ 'price_eth' ] + amount_eth * fee

        if amount_doge >= th_value:
            # DOGE 잔고 변경 
            cur_data[ 'bal2_doge' ] += cur_data[ 'trade_doge' ]
            # krw 잔고 변경 (DOGE 매매 반영 + 수수료 차감)
            cur_data[ 'bal2_krw' ] -= cur_data[ 'trade_doge' ] * cur_data[ 'price_doge' ] + amount_doge * fee

        if amount_ada >= th_value:
            # ADA 잔고 변경 
            cur_data[ 'bal2_ada' ] += cur_data[ 'trade_ada' ]
            # krw 잔고 변경 (ADA 매매 반영 + 수수료 차감)
            cur_data[ 'bal2_krw' ] -= cur_data[ 'trade_ada' ] * cur_data[ 'price_ada' ] + amount_ada * fee


        #--------------------------------------------------------------------------------------------------------------------
        # DO NOT EDIT
        #
        df.loc[ row_index, : ] = cur_data

        row_index += 1
        #--------------------------------------------------------------------------------------------------------------------




In [12]:
from tqdm.notebook import tqdm
from collections import deque

results_10_20 = deque()
results_less_10 = deque()

ratio_krw_range = np.arange(1, 0.5, -0.05)
ratio_btc_range = np.arange(0, 1, 0.1)
ratio_eth_range = np.arange(0, 1, 0.1)
ratio_doge_range = np.arange(0, 1, 0.1)
#ratio_ada_range = np.arange(0, 0.55, 0.05)
threshold_range = np.arange(0, 1, 0.01)

total_iterations = len(ratio_krw_range) * len(ratio_btc_range) #* len(ratio_eth_range) * len(ratio_doge_range) * len(threshold_range)

best_hpr_10_20 = 0.0
best_ratios_10_20 = {}
max_hpr_10_20 = 0.0
best_ratio_10_20 = None

best_hpr_less_10 = 0.0
best_ratios_less_10 = {}
max_hpr_less_10 = 0.0
best_ratio_less_10 = None

ratio_krw_tqdm = tqdm(ratio_krw_range, desc='ratio_krw', mininterval=2, maxinterval=5, leave=True)
for ratio_krw in ratio_krw_tqdm:
    ratio_krw_tqdm.set_description(f'ratio_krw = {ratio_krw}')
    ratio_btc_tqdm = tqdm(ratio_btc_range,  desc='ratio_btc', mininterval=2, maxinterval=5, leave=False)
    for ratio_btc in ratio_btc_tqdm:
        ratio_btc_tqdm.set_description(f'ratio_btc = {ratio_btc}')
        ratio_eth_tqdm = tqdm(ratio_eth_range, desc='ratio_eth', mininterval=2, maxinterval=5, position=2, leave=False)
        for ratio_eth in ratio_eth_tqdm:
            #ratio_eth_tqdm.set_description(f'ratio_eth = {ratio_eth}')
            #ratio_doge_tqdm = tqdm(ratio_doge_range, desc='ratio_doge', mininterval=2, maxinterval=5, position=3, leave=False)
            for ratio_doge in ratio_doge_range:
                #ratio_doge_tqdm.set_description(f'ratio_doge = {ratio_doge}')
                ratio_ada = 1 - (ratio_krw + ratio_btc + ratio_eth + ratio_doge)
                total_ratio = ratio_krw + ratio_btc + ratio_eth + ratio_doge + ratio_ada
                if abs(total_ratio - 1) < 1e-6 and ratio_ada >= 0:
                    #ratio_threshold_tqdm = tqdm(threshold_range, desc='threshold', mininterval=2, maxinterval=5, position=4, leave=False)
                    for threshold in threshold_range:
                        #ratio_threshold_tqdm.set_description(f'threshold = {threshold}')
                        rebalancing(df, ratio_krw, ratio_btc, ratio_eth, ratio_doge, ratio_ada, threshold)
                        hpr, mdd = trading_helper.check_performance_rebal(df)
                        if 0 < mdd <= 0.3 and hpr > 0:
                            if hpr > max_hpr_10_20:
                                max_hpr_10_20 = hpr
                                best_ratio_10_20 = (ratio_krw, ratio_btc, ratio_eth, ratio_doge, ratio_ada, threshold)
                            print(f'[Round1]ratio_krw={ratio_krw}, ratio_btc={ratio_btc}, ratio_eth={ratio_eth}, ratio_doge={ratio_doge}, ratio_ada={ratio_ada}, threshold={threshold}')
                            print( f'    HPR={hpr*100} MDD={mdd*100}' )
                            results_10_20.append((ratio_krw, ratio_btc, ratio_eth, ratio_doge, ratio_ada, threshold, hpr, mdd))
                            best_ratios_10_20[(ratio_krw, ratio_btc, ratio_eth, ratio_doge, ratio_ada, threshold)] = hpr
                       

ratio_krw:   0%|          | 0/10 [00:00<?, ?it/s]

ratio_btc:   0%|          | 0/10 [00:00<?, ?it/s]

ratio_eth:   0%|          | 0/10 [00:00<?, ?it/s]

ratio_eth:   0%|          | 0/10 [00:00<?, ?it/s]

ratio_eth:   0%|          | 0/10 [00:00<?, ?it/s]

...

ratio_eth:   0%|          | 0/10 [00:00<?, ?it/s]

ratio_eth:   0%|          | 0/10 [00:00<?, ?it/s]

ratio_eth:   0%|          | 0/10 [00:00<?, ?it/s]

ratio_eth:   0%|          | 0/10 [00:00<?, ?it/s]

ratio_eth:   0%|          | 0/10 [00:00<?, ?it/s]

In [14]:
# 가장 높은 HPR 출력
print("Best HPR (MDD 10-20):")
print("Ratios:", best_ratio_10_20)
print("HPR:", max_hpr_10_20)
print()

print("Best HPR (MDD < 10):")
print("Ratios:", best_ratio_less_10)
print("HPR:", max_hpr_less_10)
print()

Best HPR (MDD 10-20):
Ratios: (0.6999999999999997, 0.0, 0.30000000000000004, 0.0, 2.220446049250313e-16, 0.16)
HPR: 0.042420690407564576

Best HPR (MDD < 10):
Ratios: None
HPR: 0.0



In [15]:
rebalancing(df, 0.6999999999999997, 0.0, 0.30000000000000004, 0.0, 2.220446049250313e-16, 0.16)
hpr, mdd = trading_helper.check_performance_rebal(df)
print( f'HPR={hpr*100} MDD={mdd*100}' )

HPR=4.242069040756458 MDD=20.19716138604109


In [ ]:
rebalancing(df, 1.3, 0.0, 0.05, 0.1, -0.4500000000000002, 0.35000000000000003)
s7 = df['value'].copy()
hpr, mdd = trading_helper.check_performance_rebal(df)
print( f'HPR={hpr*100} MDD={mdd*100}' )


In [ ]:
# Print and store the results
print("Best HPRs (MDD 10-20):")
for ratios, hpr in best_ratios_10_20.items():
    print("Ratios:", ratios)
    print("HPR:", hpr)
    print()

print("Best HPRs (MDD < 10):")
for ratios, hpr in best_ratios_less_10.items():
    print("Ratios:", ratios)
    print("HPR:", hpr)
    print()

In [ ]:
list(results_10_20)

In [ ]:
list(results_less_10)

In [ ]:
# Sort and print the best ratios (HPR 10-20)
sorted_ratios_10_20 = sorted(best_ratios_10_20.items(), key=lambda x: x[1], reverse=True)
print("Best Ratios (HPR 10-20):")
for ratios, hpr in sorted_ratios_10_20:
    print("Ratios:", ratios)
    print("HPR:", hpr)
    print()

# Sort and print the best ratios (MDD < 10)
sorted_ratios_less_10 = sorted(best_ratios_less_10.items(), key=lambda x: x[1], reverse=True)
print("Best Ratios (MDD < 10):")
for ratios, hpr in sorted_ratios_less_10:
    print("Ratios:", ratios)
    print("HPR:", hpr)
    print()


In [ ]:
# 가장 높은 HPR (MDD 10-20)
best_ratio_10_20 = max(best_ratios_10_20, key=best_ratios_10_20.get)
best_hpr_10_20 = best_ratios_10_20[best_ratio_10_20]

# 가장 높은 HPR (MDD < 10)
best_ratio_less_10 = max(best_ratios_less_10, key=best_ratios_less_10.get)
best_hpr_less_10 = best_ratios_less_10[best_ratio_less_10]

# 결과 출력
print("Best HPR (MDD 10-20):")
print("Ratios:", best_ratio_10_20)
print("HPR:", best_hpr_10_20)
print()

print("Best HPR (MDD < 10):")
print("Ratios:", best_ratio_less_10)
print("HPR:", best_hpr_less_10)
print()
